Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside your personal course repository for the course 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [721]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
from dataclasses import dataclass, field



## The *Nim* and *Nimply* classes

In [722]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [723]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self):
        return sum(self._rows) > 0

    def __str__(self):
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property
    def rows(self) -> tuple:
        return tuple(self._rows)

    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [724]:
def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [725]:
def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [726]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)


def analyze(raw: Nim) -> dict:
    cooked = {"possible_moves": {}}
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked


def optimal(state: Nim) -> Nimply:
    analysis = analyze(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply




In [727]:
import random
from typing import Tuple
import copy

NUM_SETS = 5
TOURNAMENT_SIZE = 3
OFFSPRING_SIZE = 10
MUTATION_PROBABILITY = 0.5
POPULATION_SIZE = 20


@dataclass
class Individual:
    fitness: int
    genotype: Nim
    mutation_info: Tuple[int, int] = field(default_factory=lambda: (0, 0))


def fitness(state: Nim) -> int:
    return nim_sum(state)


def adaptive(state: Nim) -> Nimply:
    population = [
        Individual(fitness=0, genotype=copy.deepcopy(state))
        for _ in range(POPULATION_SIZE)
    ]

    def select_parent(population):
        pool = random.sample(population, TOURNAMENT_SIZE)
        champion = max(pool, key=lambda ind: ind.fitness)
        return champion

    def mutate(ind: Individual, nim_state: Nim) -> Individual:
        genotype = list(ind.genotype.rows)

        modified_set = 0
        elements_removed = 0

        for pos in range(NUM_SETS):
            elements_remaining = nim_state.rows[pos]
            num_to_remove = random.randint(0, elements_remaining)

            genotype[pos] -= num_to_remove

            if num_to_remove > 0:
                modified_set = pos
                elements_removed = num_to_remove

        new_state_nim = Nim(num_rows=len(genotype))
        for i, num in enumerate(genotype):
            for _ in range(num):
                new_state_nim.nimming(Nimply(i, 1))

        new_fitness = fitness(new_state_nim)
        mutated_individual = Individual(
            fitness=new_fitness,
            genotype=new_state_nim,
            mutation_info=(modified_set, elements_removed),
        )

        return mutated_individual

    def one_cut_xover(p1: Individual, p2: Individual) -> Individual:
        cut_point = random.randint(0, NUM_SETS - 1)

        rows_p1 = p1.genotype.rows
        rows_p2 = p2.genotype.rows

        offspring_rows = rows_p1[:cut_point] + rows_p2[cut_point:]

        offspring_nim = Nim(num_rows=len(offspring_rows))
        for i, count in enumerate(offspring_rows):
            for _ in range(count):
                offspring_nim.nimming(Nimply(i, 1))

        return Individual(fitness=0, genotype=offspring_nim)

    for _ in range(10):
        offspring = []
        for _ in range(OFFSPRING_SIZE):
            if random.random() < MUTATION_PROBABILITY:
                parent = select_parent(population)
                child = mutate(parent, state)
            else:
                parent1 = select_parent(population)
                parent2 = select_parent(population)
                child = one_cut_xover(parent1, parent2)
            offspring.append(child)

        for individual in offspring:
            individual.fitness = fitness(individual.genotype)

        population.extend(offspring)
        population.sort(key=lambda ind: ind.fitness, reverse=True)
        population = population[:POPULATION_SIZE]

        best_mutated_individual = max(offspring, key=lambda ind: ind.fitness)

    result_nimply = Nimply(
        row=best_mutated_individual.mutation_info[0],
        num_objects=best_mutated_individual.mutation_info[1],
    )
    return result_nimply

## Oversimplified match

In [729]:
from tqdm import tqdm

num_iteration = 1000
count_0 = 0
count_1 = 0

for _ in tqdm(range(num_iteration), desc="Simulating Games"):
    # logging.getLogger().setLevel(logging.INFO)  # Considera di rimuovere la modifica del livello di log se non è strettamente necessario

    strategy = (adaptive, optimal)
    nim = Nim(5)
    player = 0
    while nim:
        ply = strategy[player](nim)
        nim.nimming(ply)
        player = 1 - player
    if player == 0:
        count_0 += 1
    else:
        count_1 += 1
    # logging.info(f"status: Player {player} won!")

print(f"Number games won player_0: {count_0}, Percentage: {round(count_0 / num_iteration * 100)}%")
print(f"Number games won player_1: {count_1}, Percentage: {round(count_1 / num_iteration * 100)}%")


Simulating Games: 100%|██████████| 1000/1000 [01:42<00:00,  9.80it/s]

Number games won player_0: 660, Percentage: 66%
Number games won player_1: 340, Percentage: 34%
